# data_process & environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
!pip install -q "monai-weekly[nibabel, tqdm]"

In [ ]:
import logging
import os
import sys
import shutil
import tempfile

import matplotlib.pyplot as plt
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)

pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()

MONAI version: 1.5.dev2450
Numpy version: 1.26.4
Pytorch version: 2.5.1+cu121
MONAI flags: HAS_EXT = False, USE_COMPILED = False, USE_META_DICT = False
MONAI rev id: 0726cee7ba6c1b31f34d54fc26fe6b8db5f5c79c
MONAI __file__: /usr/local/lib/python3.10/dist-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: NOT INSTALLED or UNKNOWN VERSION.
ITK version: NOT INSTALLED or UNKNOWN VERSION.
Nibabel version: 5.3.2
scikit-image version: 0.25.0
scipy version: 1.13.1
Pillow version: 11.0.0
Tensorboard version: 2.17.1
gdown version: 5.2.0
TorchVision version: 0.20.1+cu121
tqdm version: 4.67.1
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.5
pandas version: 2.2.2
einops version: 0.8.0
transformers version: 4.47.1
mlflow version: NOT INSTALLED or UNKNOWN VERSION.
pynrrd version: NOT INSTALLED or UNKNOWN VERSION.
clearml version: NOT INSTALLED or UNKNOWN VERSION.

For details about installing the optional dependencies, please visit:
    https://docs.monai.i

In [ ]:
import os
import pandas as pd
import numpy as np
import torch

# Define the root directory
root_dir = "/content/drive/MyDrive/yufei/monaidata"

# Load the label file
label_file = os.path.join(root_dir, "LABEL0814.xlsx")
labels_df = pd.read_excel(label_file, header=None, names=["id", "label"])

# Load the train and test tables
train_file = "/content/drive/MyDrive/yufei/data/train.xlsx"
test_file = "/content/drive/MyDrive/yufei/data/test.xlsx"

train_df = pd.read_excel(train_file, header=0, names=["id", "label"])
test_df = pd.read_excel(test_file, header=0, names=["id", "label"])

# Get all image files
image_files = os.listdir(root_dir)
image_files = [f for f in image_files if f.endswith(".nii")]

# Initialize the image path and tag list
train_images, train_labels = [], []
test_images, test_labels = [], []


# Iterate over all image files
for image_file in image_files:
    # Extract patient ID
    id = "_".join(image_file.split('_')[:3])

    # Find the corresponding tag
    label_row = labels_df[labels_df['id'] == id]

    if label_row.empty:
        print(f"Warning: No matching label found for ID {id}")
    else:
        # Check if the ID belongs to train or tests
        if id in train_df['id'].values:
            # print(f"Adding {id} to train set")
            train_images.append(os.path.join(root_dir, image_file))
            train_labels.append(label_row['label'].values[0])
        elif id in test_df['id'].values:
            # print(f"Adding {id} to test set")
            test_images.append(os.path.join(root_dir, image_file))
            test_labels.append(label_row['label'].values[0])
        else:
            print(f"ID {id} not found in train or test split.")


# Debug: output warnings for empty training or test sets
if len(train_labels) == 0:
    print("Warning: No labels found for the training set.")
if len(test_labels) == 0:
    print("Warning: No labels found for the test set.")

# One-hot coding training set labels
if len(train_labels) > 0:
    train_labels_tensor = torch.as_tensor(train_labels, dtype=torch.int64)
    train_labels_one_hot = torch.nn.functional.one_hot(train_labels_tensor).float()
    # print("Train Labels (One-Hot):", train_labels_one_hot)
else:
    print("No train labels to encode.")

# One-hot coding test set tags
if len(test_labels) > 0:
    test_labels_tensor = torch.as_tensor(test_labels, dtype=torch.int64)
    test_labels_one_hot = torch.nn.functional.one_hot(test_labels_tensor).float()
    # print("Test Labels (One-Hot):", test_labels_one_hot)
else:
    print("No test labels to encode.")

print(len(train_images), len(train_labels))
print(len(test_images), len(test_labels))

187 187
80 80


In [ ]:
class CustomImageDataset(ImageDataset):
    def __getitem__(self, index):
        # Get filenames and tags
        img_file = self.image_files[index]
        label = self.labels[index]

        # Load the image and transform it
        img = self.loader(img_file)
        if self.transform is not None:
            img = self.transform(img)

        return img, label, img_file  # Return images, labels, and image filenames


In [ ]:
import pandas as pd
random_seed = 42
# Define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 32)), RandRotate90()])
val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 32))])
bz=4

# Create train dataset and dataloader
train_ds = CustomImageDataset(image_files=train_images, labels=train_labels, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=bz, shuffle=True, num_workers=2, pin_memory=pin_memory)

# Create test (validation) dataset and dataloader
val_ds = CustomImageDataset(image_files=test_images, labels=test_labels, transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=bz, shuffle=False, num_workers=2, pin_memory=pin_memory)

# Check dataset for debugging purposes
check_ds = ImageDataset(image_files=train_images, labels=train_labels, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=bz, num_workers=2, pin_memory=pin_memory)

# Print first batch for debugging
im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label, label.shape)

<class 'monai.data.meta_tensor.MetaTensor'> torch.Size([4, 1, 96, 96, 32]) tensor([0, 0, 0, 0]) torch.Size([4])


In [ ]:
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2, alpha=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha

    def forward(self, inputs, targets):

        BCE_loss = torch.nn.functional.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        if self.alpha is not None:
            at = self.alpha.gather(0, targets)
            BCE_loss = at * BCE_loss
        F_loss = ((1 - pt) ** self.gamma * BCE_loss).mean()
        return F_loss


# Model: DenseNet121

In [ ]:
import torch
from collections import Counter
from torch.utils.tensorboard import SummaryWriter
import monai

# Create DenseNet121 models, loss functions, and optimizers
model = monai.networks.nets.DenseNet121(spatial_dims=3, in_channels=1, out_channels=2).to(device)

# Weighted cross-entropy loss
loss_function = FocalLoss(alpha=torch.tensor([172, 95]).to(device))
# optimizer = torch.optim.Adam(model.parameters(), 1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

# Training and validating hyperparameters
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 50
best_train_results = []
best_val_results = []

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    train_epoch_results = []  # Store the training results for the current epoch
    for batch_data in train_loader:
        step += 1
        inputs, labels, img_names = batch_data[0].to(device), batch_data[1].to(device), batch_data[2]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        predicted_labels = outputs.argmax(dim=1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Convert raw logits to probabilities

        # Save the results of the training set for the current epoch
        for i in range(len(img_names)):
            train_epoch_results.append(
                (img_names[i], labels[i].item(), predicted_labels[i].item(), probabilities[i].cpu().tolist())
            )

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        num_correct = 0.0
        metric_count = 0
        val_epoch_results = []   # Store the validation results for the current epoch
        for val_data in val_loader:
            val_images, val_labels, val_img_names = val_data[0].to(device), val_data[1].to(device), val_data[2]
            with torch.no_grad():
                val_outputs = model(val_images)
                predicted_val_labels = val_outputs.argmax(dim=1)
                probabilities = torch.nn.functional.softmax(val_outputs, dim=1)  # Convert raw logits to probabilities
                value = torch.eq(predicted_val_labels, val_labels)
                metric_count += len(value)
                num_correct += value.sum().item()

                # Save the validation set results for the current epoch
                for i in range(len(val_img_names)):
                    val_epoch_results.append(
                        (val_img_names[i], val_labels[i].item(), predicted_val_labels[i].item(), probabilities[i].cpu().tolist())
                    )

        metric = num_correct / metric_count
        metric_values.append(metric)

       # If current accuracies are better, update training and validation results of the best models
        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_dense.pth")
            print("saved new best metric model")

             # Update the results of the best training and validation sets
            best_train_results = train_epoch_results
            best_val_results = val_epoch_results

        print(f"Current epoch: {epoch + 1} current accuracy: {metric:.4f}")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
        writer.add_scalar("val_accuracy", metric, epoch + 1)

        # Invoke the learning rate scheduler to adjust the learning rate based on validation set accuracies
        scheduler.step(metric)

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

# Print the results of the training and validation sets for the best epochs
print("\nBest epoch train results:")
for img_name, true_label, pred_label, probabilities in best_train_results:
    print(f"Train Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {probabilities}")

print("\nBest epoch validation results:")
for img_name, true_label, pred_label, probabilities in best_val_results:
    print(f"Val Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {probabilities}")

writer.close()


# Model: ResNet

In [ ]:
import monai.networks.nets as nets

# Create a ResNet model using MONAI with basic block, 3D data, input channels as 1, and output classes as 2
model = nets.ResNet(
    block="basic",            # ResNet block type
    layers=[2, 2, 2, 2],      # Number of layers in each block (ResNet18 configuration)
    block_inplanes=[64, 128, 256, 512],  # Define the number of filters in each block
    spatial_dims=3,           # 3D convolution
    n_input_channels=1,       # Input channels (for grayscale/1 channel data)
    num_classes=2,            # Output classes
    conv1_t_size=7,           # First convolution kernel size
    conv1_t_stride=2          # First convolution stride
).to(device)

loss_function = FocalLoss(alpha=torch.tensor([172, 95]).to(device))
# optimizer = torch.optim.Adam(model.parameters(), 1e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# Add learning rate scheduler
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)


# Training and validating hyperparameters
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 50
best_train_results = []
best_val_results = []

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    train_epoch_results = []
    for batch_data in train_loader:
        step += 1
        inputs, labels, img_names = batch_data[0].to(device), batch_data[1].to(device), batch_data[2]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        predicted_labels = outputs.argmax(dim=1)
        probabilities = torch.softmax(outputs, dim=1)  # Getting a probability distribution

        # Save the results of the training set for the current epoch (including image name, true label, predicted label and predicted probability)
        for i in range(len(img_names)):
            train_epoch_results.append((img_names[i], labels[i].item(), predicted_labels[i].item(), probabilities[i].detach().cpu().numpy()))

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        num_correct = 0.0
        metric_count = 0
        val_epoch_results = []  # Store the validation results for the current epoch
        for val_data in val_loader:
            val_images, val_labels, val_img_names = val_data[0].to(device), val_data[1].to(device), val_data[2]
            with torch.no_grad():
                val_outputs = model(val_images)
                predicted_val_labels = val_outputs.argmax(dim=1)
                probabilities = torch.softmax(val_outputs, dim=1)
                value = torch.eq(predicted_val_labels, val_labels)
                metric_count += len(value)
                num_correct += value.sum().item()

                # Save validation set results for the current epoch (including image name, true label, predicted label and predicted probability)
                for i in range(len(val_img_names)):
                    val_epoch_results.append((val_img_names[i], val_labels[i].item(), predicted_val_labels[i].item(), probabilities[i].detach().cpu().numpy()))

        metric = num_correct / metric_count
        metric_values.append(metric)

        # If current accuracies are better, update training and validation results of the best models
        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_res.pth")
            print("saved new best metric model")

            # Update the results of the best training and validation sets
            best_train_results = train_epoch_results
            best_val_results = val_epoch_results

        print(f"Current epoch: {epoch + 1} current accuracy: {metric:.4f}")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
        writer.add_scalar("val_accuracy", metric, epoch + 1)
        scheduler.step(metric)

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

# Print the results of the training and validation sets for the best epochs
print("\nBest epoch train results:")
for img_name, true_label, pred_label, prob in best_train_results:
    print(f"Train Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {prob}")

print("\nBest epoch validation results:")
for img_name, true_label, pred_label, prob in best_val_results:
    print(f"Val Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {prob}")

writer.close()



# VIT


In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
from monai.networks.nets import ViT
import json

# Create the ViT model
model = ViT(
    in_channels=1,
    img_size=(96, 96, 32),
    patch_size=(16, 16, 8),
    hidden_size=768,
    mlp_dim=3072,
    num_layers=12,
    num_heads=12,
    classification=True,
    num_classes=2,
    spatial_dims=3
).to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# Training and validating hyperparameters
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 50

best_train_results = []
best_val_results = []

def save_results_to_file(results, filename):
    formatted_results = []
    for img_name, true_label, pred_label, prob in results:
        formatted_results.append({
            "Image": img_name,
            "True Label": true_label,
            "Predicted Label": pred_label,
            "Probabilities": prob.tolist()
        })
    with open(filename, "w") as f:
        json.dump(formatted_results, f, indent=4)

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0

    train_epoch_results = []

    for batch_data in train_loader:
        step += 1
        inputs, labels, img_names = batch_data[0].to(device), batch_data[1].to(device), batch_data[2]  # 获取输入、标签和图像名
        optimizer.zero_grad()
        outputs, _ = model(inputs)  # Get the model's classification output
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

        # Save file name, true label, predicted label and predicted probability for each image
        predicted_labels = outputs.argmax(dim=1)
        probabilities = torch.softmax(outputs, dim=1)
        for i in range(len(img_names)):
            train_epoch_results.append((
                img_names[i],
                labels[i].item(),
                predicted_labels[i].item(),
                probabilities[i].detach().cpu().numpy()
            ))

        epoch_len = len(train_ds) // train_loader.batch_size
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        num_correct = 0.0
        metric_count = 0
        val_epoch_results = []

        for val_data in val_loader:
            val_images, val_labels, val_img_names = val_data[0].to(device), val_data[1].to(device), val_data[2]  # Get validation data and image name
            with torch.no_grad():
                val_outputs, _ = model(val_images)
                predicted_val_labels = val_outputs.argmax(dim=1)
                probabilities = torch.softmax(val_outputs, dim=1)
                value = torch.eq(predicted_val_labels, val_labels)
                metric_count += len(value)
                num_correct += value.sum().item()

                # Save the filename, true label, predicted label and predicted probability of the validation set
                for i in range(len(val_img_names)):
                    val_epoch_results.append((
                        val_img_names[i],
                        val_labels[i].item(),
                        predicted_val_labels[i].item(),
                        probabilities[i].detach().cpu().numpy()
                    ))

        metric = num_correct / metric_count
        metric_values.append(metric)

        # Best models and results preserved
        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_vit.pth")
            print("saved new best metric model")

            # Update the results of the best training and validation sets
            best_train_results = train_epoch_results
            best_val_results = val_epoch_results

        print(f"Current epoch: {epoch + 1} current accuracy: {metric:.4f}")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
        writer.add_scalar("val_accuracy", metric, epoch + 1)

# Print the results of the training and validation sets for the best epochs
print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")


save_results_to_file(best_train_results, "best_train_results.json")
save_results_to_file(best_val_results, "best_val_results.json")

print("\nBest epoch train results:")
for img_name, true_label, pred_label, prob in best_train_results:
    print(f"Train Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {prob}")

print("\nBest epoch validation results:")
for img_name, true_label, pred_label, prob in best_val_results:
    print(f"Val Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {prob}")

writer.close()


# ResNetFeatures+XGboost/FC

In [ ]:
import torch
import monai
import numpy as np
import xgboost as xgb

# Initialize ResNetFeatures
model = monai.networks.nets.ResNetFeatures(
    model_name="resnet18",  # It could be resnet10, resnet18, resnet34, etc. #
    pretrained=True,
    spatial_dims=3,         # 3D data
    in_channels=1
).to(device)


model.eval()

# Extract the features
def extract_features(model, loader):
    features_list = []
    labels_list = []

    with torch.no_grad():
        for batch_data in loader:
            inputs, labels = batch_data[0].to(device), batch_data[1].to(device)

            features = model(inputs)  # Get a list of features
            final_features = features[-1]  # Select the last layer of features

            # Spread features to 2D: [batch_size, n_features]
            final_features = final_features.view(final_features.size(0), -1)

            features_list.append(final_features.cpu().numpy())
            labels_list.append(labels.cpu().numpy())

    features_array = np.concatenate(features_list, axis=0)
    labels_array = np.concatenate(labels_list, axis=0)

    return features_array, labels_array


XGBOOST

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
import xgboost as xgb

# Initialize the classifier
xgb_classifier = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    use_label_encoder=False
)

svm_classifier = SVC(probability=True)  # Using SVM classifiers
logreg_classifier = LogisticRegression()  # Using logistic regression classifiers

# Soft voting fusion of three classifiers
voting_classifier = VotingClassifier(
    estimators=[
        ('xgb', xgb_classifier),
        ('svm', svm_classifier),
        ('logreg', logreg_classifier)
    ],
    voting='soft'  # Soft voting (using probability weighting)
)

# Extract features
train_features, train_labels = extract_features(model, train_loader)
val_features, val_labels = extract_features(model, val_loader)

# Training fusion models
voting_classifier.fit(train_features, train_labels)

# Predictions on validation sets
val_predictions = voting_classifier.predict(val_features)
val_probabilities = voting_classifier.predict_proba(val_features)

# Calculation accuracy
accuracy = np.mean(val_predictions == val_labels)
print(f"Validation Accuracy with Soft Voting: {accuracy:.4f}")

# Print the predictions for each sample
for i in range(len(val_labels)):
    true_label = val_labels[i]
    predicted_label = val_predictions[i]
    probability = val_probabilities[i]
    print(f"Sample {i}: True Label: {true_label}, Predicted Label: {predicted_label}, Probabilities: {probability}")


FC layer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define simple classification networks
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)  #  # flatten to [batch_size, num_features] (batch size, number of features)
        return self.fc(x)

# Functions for training classification networks
def train_classifier(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=20):
    model.to(device)
    best_val_acc = 0.0
    best_epoch = -1
    best_train_predictions = []
    best_val_predictions = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        # train
        for batch_data in train_loader:
            inputs, labels, img_names = batch_data[0], batch_data[1], batch_data[2]  # 包括图像路径名
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward propagation
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backpropagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

             # Calculate training accuracy
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)

        avg_train_loss = running_loss / len(train_loader)
        train_acc = correct_train / total_train
        print(f"Epoch [{epoch + 1}/{num_epochs}] Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_acc:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct_val = 0
        total_val = 0
        train_predictions = []
        val_predictions = []

        with torch.no_grad():
            for batch_data in train_loader:
                inputs, labels, img_names = batch_data[0], batch_data[1], batch_data[2]
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                probabilities = torch.softmax(outputs, dim=1)
                _, preds = torch.max(outputs, 1)

                for i in range(inputs.size(0)):
                    true_label = labels[i].item()
                    pred_label = preds[i].item()
                    prob = probabilities[i].cpu().numpy()  # Probability of obtaining each category
                    train_predictions.append((img_names[i], true_label, pred_label, prob))

            for batch_data in val_loader:
                inputs, labels, img_names = batch_data[0], batch_data[1], batch_data[2]
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()

                probabilities = torch.softmax(outputs, dim=1)
                _, preds = torch.max(outputs, 1)
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)

                for i in range(inputs.size(0)):
                    true_label = labels[i].item()
                    pred_label = preds[i].item()
                    prob = probabilities[i].cpu().numpy() # Probability of obtaining each category
                    val_predictions.append((img_names[i], true_label, pred_label, prob))

        avg_val_loss = val_loss / len(val_loader)
        val_acc = correct_val / total_val
        print(f"Epoch [{epoch + 1}/{num_epochs}] Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

        # Preservation of the best models
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch + 1
            best_train_predictions = train_predictions
            best_val_predictions = val_predictions
            torch.save(model.state_dict(), "best_classifier.pth")
            print("Best model saved.")

    print(f"Training completed. Best Validation Accuracy: {best_val_acc:.4f} at Epoch {best_epoch}")

    # Print the best training and validation results
    print("Best Training Predictions:")
    for img_name, true_label, pred_label, prob in best_train_predictions:
        print(f"Train Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {prob}")

    print("Best Validation Predictions:")
    for img_name, true_label, pred_label, prob in best_val_predictions:
        print(f"Validation Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {prob}")

# Initialize the classification network
input_size = 96 * 96 * 32  # Assuming the input features are flattened
num_classes = len(np.unique(train_labels))  # categories based on labels
classifier = SimpleClassifier(input_size=input_size, num_classes=num_classes)

# Define loss functions and optimizers
# criterion = nn.CrossEntropyLoss()
criterion = FocalLoss(alpha=torch.tensor([172, 95]).to(device))
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Assume train_loader and val_loader are defined and device is device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train the classification network
train_classifier(classifier, train_loader, val_loader, criterion, optimizer, device, num_epochs=50)


# MLP


In [ ]:
class SimpleMLP(torch.nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleMLP, self).__init__()
        # Define two fully connected layers
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.relu = torch.nn.ReLU()  # Activation function
        self.fc2 = torch.nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Spread inputs to accommodate fully connected layers
        x = torch.flatten(x, 1)  # Flatten the input, keeping batch size
        x = self.fc1(x)  # Input -> first fully connected layer
        x = self.relu(x)  # Apply the ReLU activation function
        x = self.fc2(x)  # Layer 1 output -> Layer 2 fully connected layer
        return x

# Define input parameters
input_size = 96 * 96 * 32
hidden_size = 128  # of neurons in the hidden layer, can be adjusted as needed
num_classes = 2  # of categories output


In [ ]:
# Create the SimpleMLP model
model = SimpleMLP(input_size=input_size, hidden_size=hidden_size, num_classes=num_classes).to(device)


loss_function = torch.nn.CrossEntropyLoss()
# loss_function = torch.nn.BCEWithLogitsLoss()  # also works with this data

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 50

# Track best epoch results
best_train_results = []
best_val_results = []

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    train_epoch_results = []  # Store current epoch training results

    for batch_data in train_loader:
        step += 1
        inputs, labels, img_names = batch_data[0].to(device), batch_data[1].to(device), batch_data[2]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        predicted_labels = outputs.argmax(dim=1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Convert raw logits to probabilities

        # Save current epoch training results
        for i in range(len(img_names)):
            train_epoch_results.append(
                (img_names[i], labels[i].item(), predicted_labels[i].item(), probabilities[i].cpu().tolist())
            )

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        num_correct = 0.0
        metric_count = 0
        val_epoch_results = []  # Store current epoch validation results

        for val_data in val_loader:
            val_images, val_labels, val_img_names = val_data[0].to(device), val_data[1].to(device), val_data[2]
            with torch.no_grad():
                val_outputs = model(val_images)
                predicted_val_labels = val_outputs.argmax(dim=1)
                probabilities = torch.nn.functional.softmax(val_outputs, dim=1)  # Convert raw logits to probabilities
                value = torch.eq(predicted_val_labels, val_labels)
                metric_count += len(value)
                num_correct += value.sum().item()

                # Save current epoch validation results
                for i in range(len(val_img_names)):
                    val_epoch_results.append(
                        (val_img_names[i], val_labels[i].item(), predicted_val_labels[i].item(), probabilities[i].cpu().tolist())
                    )

        metric = num_correct / metric_count
        metric_values.append(metric)

        # If current accuracy is the best, update best model results
        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_dense.pth")
            print("saved new best metric model")

            # Update best training and validation results
            best_train_results = train_epoch_results
            best_val_results = val_epoch_results

        print(f"Current epoch: {epoch + 1} current accuracy: {metric:.4f}")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
        writer.add_scalar("val_accuracy", metric, epoch + 1)

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

# Print best epoch results
print("\nBest epoch train results:")
for img_name, true_label, pred_label, probabilities in best_train_results:
    print(f"Train Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {probabilities}")

print("\nBest epoch validation results:")
for img_name, true_label, pred_label, probabilities in best_val_results:
    print(f"Val Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {probabilities}")

writer.close()


# CNN

In [ ]:
class SimpleCNN(torch.nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # Define a three-layer convolutional network with a single channel (1 channel) for input and num_classes categories for output
        self.conv1 = torch.nn.Conv3d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv3d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

        # Define pooling layers for reducing the spatial dimension of the feature map
        self.pool = torch.nn.MaxPool3d(kernel_size=2, stride=2, padding=0)

       # Fully connected layer, inputs are spread from convolutional layer and passed to MLP
        self.fc1 = torch.nn.Linear(64 * 12 * 12 * 4, 128)
        self.fc2 = torch.nn.Linear(128, num_classes)

        # Activation function
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        # After three layers of convolution and pooling
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)

        x = torch.flatten(x, 1)  # Flatten the input to [batch_size, features]

        # Full connectivity layer
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# of classification categories defined
num_classes = 2


In [ ]:
# Create the SimpleCNN model
model = SimpleCNN(num_classes=num_classes).to(device)

# Loss function and optimizer remain the same
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter()
max_epochs = 50

# Track best epoch results
best_train_results = []
best_val_results = []

for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    train_epoch_results = []  # Store current epoch training results

    for batch_data in train_loader:
        step += 1
        inputs, labels, img_names = batch_data[0].to(device), batch_data[1].to(device), batch_data[2]
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_len = len(train_ds) // train_loader.batch_size
        writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        predicted_labels = outputs.argmax(dim=1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Convert raw logits to probabilities

        # Save current epoch training results
        for i in range(len(img_names)):
            train_epoch_results.append(
                (img_names[i], labels[i].item(), predicted_labels[i].item(), probabilities[i].cpu().tolist())
            )

    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        model.eval()
        num_correct = 0.0
        metric_count = 0
        val_epoch_results = []  # Store current epoch validation results

        for val_data in val_loader:
            val_images, val_labels, val_img_names = val_data[0].to(device), val_data[1].to(device), val_data[2]
            with torch.no_grad():
                val_outputs = model(val_images)
                predicted_val_labels = val_outputs.argmax(dim=1)
                probabilities = torch.nn.functional.softmax(val_outputs, dim=1)  # Convert raw logits to probabilities
                value = torch.eq(predicted_val_labels, val_labels)
                metric_count += len(value)
                num_correct += value.sum().item()

                # Save current epoch validation results
                for i in range(len(val_img_names)):
                    val_epoch_results.append(
                        (val_img_names[i], val_labels[i].item(), predicted_val_labels[i].item(), probabilities[i].cpu().tolist())
                    )

        metric = num_correct / metric_count
        metric_values.append(metric)

        # If current accuracy is the best, update best model results
        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), "best_metric_model_classification3d_dense.pth")
            print("saved new best metric model")

            # Update best training and validation results
            best_train_results = train_epoch_results
            best_val_results = val_epoch_results

        print(f"Current epoch: {epoch + 1} current accuracy: {metric:.4f}")
        print(f"Best accuracy: {best_metric:.4f} at epoch {best_metric_epoch}")
        writer.add_scalar("val_accuracy", metric, epoch + 1)

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")

# Print best epoch results
print("\nBest epoch train results:")
for img_name, true_label, pred_label, probabilities in best_train_results:
    print(f"Train Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {probabilities}")

print("\nBest epoch validation results:")
for img_name, true_label, pred_label, probabilities in best_val_results:
    print(f"Val Image: {img_name}, True Label: {true_label}, Predicted Label: {pred_label}, Probabilities: {probabilities}")

writer.close()